In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.cross_validation import cross_val_score
from scipy.stats import skew

''' Calculates RMSE '''
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

''' Pre-processing '''
train = pd.read_csv('/Users/shammakabir/Downloads/train.csv')
test = pd.read_csv('/Users/shammakabir/Downloads/test.csv')
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

# Log transform the classification set
train["SalePrice"] = np.log1p(train["SalePrice"])
prices = pd.DataFrame(train["SalePrice"])

# Log transform skewed numeric features
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

# Convert categorical features into dummy variables
all_data = pd.get_dummies(all_data)

# Fill empty cells with the mean of the column
all_data = all_data.fillna(all_data.mean())

# Create matrices for sklearn
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

''' Part 2 '''
# Train a ridge regression model
alphas = [0.01, 0.05, 0.1, 0.5, 1, 2, 4, 8, 10, 15, 30, 45, 60]
model_ridge = [rmse_cv(Ridge(alpha=alpha, solver='svd', tol=0.001)).mean() for alpha in alphas]
model_ridge = pd.Series(model_ridge, index = alphas)
model_ridge.min()

# Train a lasso regression model
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, y)
rmse_cv(model_lasso).mean()
coef = pd.Series(model_lasso.coef_, index = X_train.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")
#let's look at the residuals as well:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":model_lasso.predict(X_train), "true":y})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

/Users/shammakabir/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Lasso picked 110 variables and eliminated the other 178 variables


In [3]:
import xgboost as xgb

In [4]:
dtrain = xgb.DMatrix(X_train, label = y)
dtest = xgb.DMatrix(X_test)

params = {"max_depth":2, "eta":0.1}
model = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)

Will train until cv error hasn't decreased in 100 rounds.
/Users/shammakabir/anaconda3/lib/python3.5/site-packages/xgboost/training.py:272: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  idset = [randidx[(i * kstep): min(len(randidx), (i + 1) * kstep)] for i in range(nfold)]


In [6]:
model.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()

In [6]:
model_xgb = xgb.XGBRegressor(n_estimators=500, max_depth=2, learning_rate=0.1) #the params were tuned using xgb.cv
model_xgb.fit(X_train, y)
error = rmse_cv(model_xgb).mean()
print(error)

#LOWEST ERROR: 0.124044090248

0.124044090248


In [7]:
#xgb_preds = np.expm1(model_xgb.predict(X_test))
#lasso_preds = np.expm1(model_lasso.predict(X_test))

In [8]:
#predictions = pd.DataFrame({"xgb":xgb_preds, "lasso":lasso_preds})
#predictions.plot(x = "xgb", y = "lasso", kind = "scatter")

In [9]:
#preds = 0.8*lasso_preds + 0.2*xgb_preds

In [10]:
#solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
#solution.to_csv("/Users/shammakabir/Desktop/xgbsolomg.csv", index = False)

#0.12087